<div class="alert alert-block alert-warning">
<b>REQUIREMENT:</b>
    Before proceeding, install or update your
    <b><a class="divlink" href="https://stenv.readthedocs.io/en/latest/">
    stenv
    </a></b>
     distribution. <b>stenv</b> is the replacement for AstroConda, which is unsupported as of February 2023.
</div>

# Using ACS Polarization Tools

## Introduction

***

In December 2020, the ACS Team made available a Python module called `polarization_tools` in the `acstools` package to facilitate data analysis of ACS polarization data. This package contains both tables of calibration terms and class methods for polarization calculations.

Note that this example does not use imaging data (i.e., FITS files). Where necessary, we will indicate values that come from ACS FITS files.
</div>

### This tutorial will show you how to...

#### [Retrieve Polarization Calibration Coefficients](#_coeffs) 

* Get the latest calibration coefficients that are used in `acstools`

#### [Calculate Polarization Properties](#_properties)

* Use the `acstools.polarization_tools` module to compute:
    * Stokes parameters
    * Fractional polarization
    * Electric field vector position angle

## Imports

***

Here we list the Python packages used in this notebook. Links to the documentation for each module is provided for convenience.

| Package Name     | module           | docs          | used for    |
|------------------|:-----------------|:-------------:|:------------|
|`acstools`       |`polarization_tools`| <a class="divlink" href="https://acstools.readthedocs.io/en/latest/polarization_tools.html">link</a>| polarization information and methods 

In [1]:
from acstools import polarization_tools

### Retrieve Polarization Calibration Coefficients<a id="_coeffs"></a>

***

The `polarization_tools` module contains all of the necessary calibration terms from the tables in section 5.3 of the ACS Data Handbook (see tables 5.6 and 5.7). These coefficients are stored in YAML files in `acstools` and may be retrieved as `astropy.Table` tables:

In [2]:
polarizer_tables = polarization_tools.PolarizerTables.from_package_data()

# print the ACS/WFC polarizer efficiencies
polarizer_tables.wfc_efficiency

filter,pol0,pol60,pol120
str5,float64,float64,float64
F475W,1.4303,1.4717,1.4269
F606W,1.3314,1.3607,1.3094
F775W,0.9965,1.0255,1.0071


In [3]:
# print the average ACS/WFC cross-polarization leak corrections
polarizer_tables.wfc_transmission

filter,t_para,t_perp,correction
str5,float64,float64,float64
F475W,0.4239276798513098,0.00015240583841551956,1.000719276691027
F606W,0.5156734594049419,5.5908749369641956e-05,1.000216861312415
F775W,0.6040891283746557,0.07367364117759412,1.2777959654493372


In addition, the tables contain metadata that describe their derivation:

In [4]:
# print the ACS/WFC cross-polarization leak correction metadata
polarizer_tables.wfc_transmission.meta

{'description': 'WFC filters use MJD corresponding to 2020-01-01. HRC filters use MJD corresponding to 2007-01-01.'}

### Calculate Polarization Properties<a id="_properties"></a>

***

The `acstools.polarization_tools.Polarizer` class contains methods for calculating the polarization properties of a source given ACS photometry as input. The inputs may be either in electrons or electrons/second, as long as all three (for each POL0, POL60, and POL120) are consistently in the same units. In addition, the class can accept either a single float value for each photometric measurement or a `numpy` array. This has the advantage that the class can construct Stokes images from the ACS photometry.

For this example, we will use photometry measurements of the polarizated calibration star Vela 1-81 (an OB supergiant). The observations were taken using the ACS/WFC detector and by crossing the polarization filters with the F606W filter. The average PA_V3 value from the primary headers of the FITS files is 294.41272 degrees.

In [5]:
# Measured photometry from ACS/WFC data
pol0 = 60603.96
pol60 = 62382.70
pol120 = 67898.63

# PA_V3 angle
pa_v3 = 294.41272

pol_data = polarization_tools.Polarization(pol0, pol60, pol120, 'F606W', 'WFC', pa_v3)
pol_data.calc_stokes()
pol_data.calc_polarization()
print(f'Stokes I = {pol_data.stokes_i}, Stokes Q = {pol_data.stokes_q}, Stokes U = {pol_data.stokes_u}')
print(f'Fractional polarization = {pol_data.polarization:0.2%}, Position Angle = {pol_data.angle}')

Stokes I = 169652.47890399996, Stokes Q = -8276.254216000001, Stokes U = -4644.582265627395
Fractional polarization = 5.60%, Position Angle = 180.8631543698736 deg


Notice that we did not provide any of the calibration coefficients from the Data Handbook or from the [Retrieve Polarization Calibration Coefficients](#_coeffs) section. This is because the `Polarization` class retrieves the values using the spectral filter and detector information. These calibration values may be overwritten manually by setting the appropriate attributes:

In [6]:
# print the efficiency coefficients cross-polarization leak term used
print(f'c0 = {pol_data.c0}, c60 = {pol_data.c60}, c120 = {pol_data.c120}, cross-leak = {pol_data.transmission_correction}')

c0 = 1.3314, c60 = 1.3607, c120 = 1.3094, cross-leak = 1.000216861312415


### For more help:

More details may be found on the [ACS website](http://www.stsci.edu/hst/instrumentation/acs) and in the [ACS Instrument](https://hst-docs.stsci.edu/display/ACSIHB) and [Data Handbooks](http://www.stsci.edu/files/live/sites/www/files/home/hst/instrumentation/acs/documentation/other-documents/_documents/acs_dhb.pdf).

Please visit the [HST Help Desk](http://hsthelp.stsci.edu). Through the help desk portal, you can explore the *HST* Knowledge Base and request additional help from experts.